In [0]:
%pip install pymongo

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.0/670.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 19.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
import os
import json
import pymongo
import pyspark.pandas as pd  # This uses Koalas that is included in PySpark version 3.2 or newer.
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

In [0]:
dbutils.fs.rm("/FileStore/tables/project_data/bronze",True)
dbutils.fs.rm("/FileStore/tables/project_data/silver",True)
dbutils.fs.rm("/FileStore/tables/project_data/gold",True)

False

In [0]:
# Azure MySQL Server Connection Information ###################
jdbc_hostname = "<rmg7jb>.mysql.database.azure.com"
jdbc_port = 3306
src_database = "chinook_dw"

connection_properties = {
  "user" : "rmg7jb",
  "password" : "Passw0rd123!",
  "driver" : "org.mariadb.jdbc.Driver"
}

# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "cluster0.6kd78gj"
atlas_database_name = "Chinook"
atlas_user_name = "rmg7jb"
atlas_password = "Passw0rd123"


In [0]:
##################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
##################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe


In [0]:
%sql
DROP DATABASE IF EXISTS chinook_dlh CASCADE;

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS chinook_dlh
COMMENT "Gunn Final Project Database"
LOCATION "dbfs:/FileStore/lab_data/chinook_dlh"
WITH DBPROPERTIES (contains_pii = true, purpose = "Gunn Final Project");

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://rmg7jb.mysql.database.azure.com:3306/chinook_dw", --Replace with your Server Name
  dbtable "dim_date",
  user "rmg7jb",    --Replace with your User Name
  password "Passw0rd123!"  --Replace with you password
)

In [0]:
%sql
USE DATABASE chinook_dlh;

CREATE OR REPLACE TABLE chinook_dlh.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/lab_data/chinook_dlh/dim_date"
AS SELECT * FROM view_date

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED chinook_dlh.dim_date;

col_name,data_type,comment
date_key,int,null
full_date,varchar(65535),null
date_name,varchar(65535),null
date_name_us,varchar(65535),null
date_name_eu,varchar(65535),null
day_of_week,int,null
day_name_of_week,varchar(65535),null
day_of_month,int,null
day_of_year,int,null
weekday_weekend,varchar(65535),null


In [0]:
%sql
SELECT * FROM chinook_dlh.dim_date LIMIT 5

date_key,full_date,date_name,date_name_us,date_name_eu,day_of_week,day_name_of_week,day_of_month,day_of_year,weekday_weekend,week_of_year,month_name,month_of_year,is_last_day_of_month,calendar_quarter,calendar_year,calendar_year_month,calendar_year_qtr,fiscal_month_of_year,fiscal_quarter,fiscal_year,fiscal_year_month,fiscal_year_qtr
20000101,2000-01-01,2000/01/01,01/01/2000,01/01/2000,7,Saturday,1,1,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000102,2000-01-02,2000/01/02,01/02/2000,02/01/2000,1,Sunday,2,2,Weekend,52,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000103,2000-01-03,2000/01/03,01/03/2000,03/01/2000,2,Monday,3,3,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000104,2000-01-04,2000/01/04,01/04/2000,04/01/2000,3,Tuesday,4,4,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3
20000105,2000-01-05,2000/01/05,01/05/2000,05/01/2000,4,Wednesday,5,5,Weekday,1,January,1,N,1,2000,2000-01,2000Q1,7,3,2000,2000-07,2000Q3


In [0]:
%sql
-- Create a Temporary View named "view_tracks"
CREATE OR REPLACE TEMPORARY VIEW view_tracks
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://rmg7jb.mysql.database.azure.com:3306/chinook_dw", --Replace with your Server Name
  dbtable "dim_tracks",
  user "rmg7jb",    --Replace with your User Name
  password "Passw0rd123!"  --Replace with you password
)


In [0]:
%sql
USE DATABASE chinook_dlh;
-- Create a new table named "chinook_dlh.dim_tracks" using data from the view named "view_tracks"
CREATE OR REPLACE TABLE chinook_dlh.dim_tracks
COMMENT "Tracks Dimension Table"
LOCATION "dbfs:/FileStore/lab_data/chinook_dlh/dim_tracks"
AS SELECT * FROM view_tracks

num_affected_rows,num_inserted_rows


In [0]:
%sql
DESCRIBE EXTENDED chinook_dlh.dim_tracks;

col_name,data_type,comment
track_key,bigint,null
track_id,bigint,null
Name,varchar(65535),null
album_id,bigint,null
media_type_id,bigint,null
genre_id,bigint,null
Composer,varchar(65535),null
Milliseconds,bigint,null
Bytes,bigint,null
UnitPrice,double,null


In [0]:
%scala
import com.mongodb.spark._

val userName = "rmg7jb"
val pwd = "Passw0rd123"
val clusterName = "cluster0.6kd78gj"
val atlas_uri = s"mongodb+srv://$userName:$pwd@$clusterName.mongodb.net/?retryWrites=true&w=majority"
val df_genre = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("spark.mongodb.input.uri", atlas_uri)
.option("database", "Chinook")
.option("collection", "genre").load()
display(df_genre)

GenreId,Name,_id
1,Rock,List(663d5b14c7bc0a91ded908fc)
2,Jazz,List(663d5b14c7bc0a91ded908fd)
3,Metal,List(663d5b14c7bc0a91ded908fe)
4,Alternative & Punk,List(663d5b14c7bc0a91ded908ff)
5,Rock And Roll,List(663d5b14c7bc0a91ded90900)
6,Blues,List(663d5b14c7bc0a91ded90901)
7,Latin,List(663d5b14c7bc0a91ded90902)
8,Reggae,List(663d5b14c7bc0a91ded90903)
9,Pop,List(663d5b14c7bc0a91ded90904)
10,Soundtrack,List(663d5b14c7bc0a91ded90905)


In [0]:
%scala
df_genre.write.format("delta").mode("overwrite").saveAsTable("chinook_dlh.dim_genre")

In [0]:
%sql
DESCRIBE EXTENDED chinook_dlh.dim_genre

col_name,data_type,comment
GenreId,int,null
Name,string,null
_id,struct,null
,,
# Delta Statistics Columns,,
Column Names,"GenreId, Name, _id",
Column Selection Method,first-32,
,,
# Detailed Table Information,,
Catalog,spark_catalog,


In [0]:
%sql
-- Create a Temporary View named "view_customers"
CREATE OR REPLACE TEMPORARY VIEW view_customers
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://rmg7jb.mysql.database.azure.com:3306/chinook_dw", --Replace with your Server Name
  dbtable "dim_Customers",
  user "rmg7jb",    --Replace with your User Name
  password "Passw0rd123!"  --Replace with you password
)

In [0]:
%sql
USE DATABASE chinook_dlh;
CREATE OR REPLACE TABLE chinook_dlh.dim_customers
COMMENT "Customers Dimension Table"
LOCATION "dbfs:/FileStore/lab_data/chinook_dlh/dim_customers"
AS SELECT * FROM view_customers

num_affected_rows,num_inserted_rows


In [0]:
%sql
USE chinook_dlh;
SHOW TABLES

database,tableName,isTemporary
chinook_dlh,dim_customers,false
chinook_dlh,dim_date,false
chinook_dlh,dim_genre,false
chinook_dlh,dim_tracks,false
,view_customers,true
,view_date,true
,view_tracks,true


In [0]:
filePath = "dbfs:/FileStore/tables/project_data"

In [0]:
(spark.readStream
 .format("cloudFiles")
 .option("cloudFiles.format", "json")
 .option("cloudFiles.schemaLocation", "dbfs:/FileStore/tables/project_data/bronze")
 .option("cloudFiles.inferColumnTypes", "true")
 .option("multiLine", "true")
 .load(filePath)
 .createOrReplaceTempView("orders_raw_tempview"))


In [0]:
%sql
/* Add Metadata for Traceability */
CREATE OR REPLACE TEMPORARY VIEW orders_bronze_tempview AS (
  SELECT customer_id,id,invoice_date,invoice_line_id,quantity,track_id,unit_price,invoice_id, current_timestamp() receipt_time, input_file_name() source_file
  FROM orders_raw_tempview
)

In [0]:
%sql
SELECT * FROM orders_bronze_tempview LIMIT 5

In [0]:
(spark.table("orders_bronze_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", "dbfs:/FileStore/tables/project_data/bronze")
      .outputMode("append")
      .table("fact_orders_bronze"))

In [0]:
(spark.readStream
  .table("fact_orders_bronze")
  .createOrReplaceTempView("orders_silver_tempview"))

In [0]:
%sql
DESCRIBE EXTENDED orders_silver_tempview

col_name,data_type,comment
customer_id,bigint,null
id,bigint,null
invoice_date,bigint,null
invoice_line_id,bigint,null
quantity,bigint,null
track_id,bigint,null
unit_price,double,null
invoice_id,bigint,null
receipt_time,timestamp,null
source_file,string,null


In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW fact_orders_silver_tempview AS (
  SELECT o.id,
      o.invoice_id,
      o.invoice_line_id,
      o.track_id,
      o.customer_id,
      o.invoice_date,
      o.unit_price,
      o.quantity,
      c.FirstName,
      c.LastName,
      c.Company,
      c.Address,
      c.City,
      c.State,
      c.Country,
      c.PostalCode,
      c.Phone,
      c.Fax,
      c.Email,
      c.SupportRepId,
      t.track_key,
      t.Name,
      t.album_id,
      t.media_type_id,
      t.genre_id,
      t.Composer,
      t.Milliseconds,
      t.Bytes,
      t.UnitPrice,
      d.date_key

  FROM orders_silver_tempview AS o
  INNER JOIN chinook_dlh.dim_customers AS c
  ON c.customer_id = o.customer_id
  INNER JOIN chinook_dlh.dim_tracks AS t
  ON t.track_id = o.track_id
  LEFT OUTER JOIN chinook_dlh.dim_date AS d
  ON d.date_key = o.invoice_date
)

In [0]:
%sql
SELECT * FROM fact_orders_silver_tempview LIMIT 5

In [0]:
(spark.table("fact_orders_silver_tempview")
      .writeStream
      .format("delta")
      .option("checkpointLocation", "dbfs:/FileStore/tables/project_data/silver/_checkpoint")
      .outputMode("append")
      .table("fact_orders_silver"))
      

In [0]:
%sql
CREATE OR REPLACE TABLE chinook_dlh.fact_orders_by_customer_gold AS (
  SELECT customer_id AS CustomerID
    , LastName
    , FirstName
    , unit_price * quantity AS Amount
  FROM chinook_dlh.fact_orders_silver
  GROUP BY CustomerID, LastName, FirstName, quantity, unit_price
  ORDER BY Amount DESC);

SELECT * FROM chinook_dlh.fact_orders_by_customer_gold;

CustomerID,LastName,FirstName,Amount
59,Srivastava,Puja,1.99
51,Johansson,Joakim,1.99
57,Rojas,Luis,1.99
19,Goyer,Tim,1.99
24,Ralston,Frank,1.99
44,Hämäläinen,Terhi,1.99
40,Lefebvre,Dominique,1.99
42,Girard,Wyatt,1.99
4,Hansen,Bjørn,1.99
43,Mercier,Isabelle,1.99
